# ESO 137-001 - HII SFRs

This notebook is to tabulate the SFR for the HII regions in ESO 137-001. For further information
see my `Tabulating ESO 137-001 SFR` note in Obsidian.

## Imports

In [ ]:
# Python Imports
from pathlib import Path
from functools import reduce
from tqdm.notebook import tqdm

In [ ]:
# Numerical Imports
import numpy as np
from scipy.interpolate import interp1d
import pandas as pd

In [ ]:
# Astropy Collaboration Imports
from astropy import units as u, uncertainty as unc
from astropy.io import fits
from astropy.wcs import WCS
from astropy.table import QTable
from astropy.coordinates import SkyCoord

import regions

from pysynphot import ObsBandpass

In [ ]:
# Plotting Imports
from matplotlib import pyplot as plt

In [ ]:
# Local Imports
from ssp import Starburst99Spectrum, CloudySpectrum

## Functions

In [ ]:
# Check to see if an array is sorted without repeats
def is_sorted_no_repeats(arr):
    return np.all(arr[:-1] < arr[1:])

In [ ]:
def cal00(wave, ebv=0.0, rvp=4.05):
    '''Calculates reddening according to Calzetti's 2000 Paper
    '''

    # Setup k
    wave = np.asanyarray(wave)
    k = np.empty_like(wave)

    # Get Waves below/above 0.63 microns
    msk = wave < 0.63
    waveLow  = wave[msk]
    waveHigh = wave[~msk]

    # Get k values below 0.63 microns
    if msk.any():
        k[msk] = rvp + 2.659*(-2.156 + 1.509/waveLow - 0.198/waveLow**2 +
                              0.011/waveLow**3)

    # Get k values above 0.63 microns
    if not msk.all():
        k[~msk] = rvp + 2.659*(-1.857 + 1.04/waveHigh)

    return k, ebv * k

In [ ]:
def unc_euclidean_inds(
    eX: unc.Distribution, eY: unc.Distribution,
    tX: np.ndarray, tY: np.ndarray, yrs: np.ndarray
):

    # Get Inds
    inds, yrEsts = [], []
    for x, y in tqdm(tuple(zip(eX, eY))):

        # Get the Distance of Each MC Point to Each Track Point
        dists = np.sqrt((x-tX)**2 + (y-tY)**2)

        # Get the Matching Index (Index of Minimum distance)
        # for Each MC Point to the track
        i = np.argmin(dists.distribution, axis=0)
        inds.append(i)

        # Get the Year Ests
        yrEsts.append(yrs[i])

    # Return Dists
    return np.array(inds), unc.Distribution(yrEsts) * yrs.unit

In [ ]:
# SFR Scalar
HA_SFR_SCLR = 10**-41.257 * u.M_sun / u.yr / (u.erg/u.s)
# Ha SFR
@u.quantity_input(luminHA=u.erg/u.s)
def ha_sfr(luminHA) -> u.M_sun / u.yr:
    return HA_SFR_SCLR * luminHA

## Notebook Setup

In [ ]:
# Seed RNG
np.random.seed(0)

# Numpy Warnings
_ = np.seterr(invalid='ignore', divide='ignore')

In [ ]:
# Directories
TOP_DIR = Path('../../').resolve()
IMG_DIR = TOP_DIR / 'Images/ProcessedImages/HST/Drizzled'
CAT_DIR = TOP_DIR / 'Science/SExtractor/Catalogs'
REG_DIR = Path(IMG_DIR / '../DS9').resolve()
OLD_REG_DIR = TOP_DIR / 'DoctoralWork/Images/MAST_DATA/WaldronPipeline/DS9'
OUT_DIR = Path('Results').resolve()

# Files
GAIA_REG_FILE  = TOP_DIR / 'Data/GAIA/ESO137-001-GAIA-AlignmentStars-Coordinates.reg'
OLD_475_IMG_FN = TOP_DIR / 'DoctoralWork/Images/MAST_DATA/WaldronPipeline/CleanedImages/F475W_drc_chip_cleaned.fits'
MUSE_FN = TOP_DIR / 'DoctoralWork/Data/ESO_137-001/MUSE/HII_EW_Properties-updated-corrected-cleaned.json'
SB99_FN = TOP_DIR / 'DoctoralWork/Data/Starburst99/Sims/TailSim0_Genv00_014/TailSim0_Genv00_014.spectrum1'
CLDY_GLOB = TOP_DIR / 'DoctoralWork/Data/Cloudy/HII/Genv00_014_10Myr/rin_10pc/**/*.cont'

In [ ]:
# Image File Names
FILTERS = [275, 475, 814]
IMG_FNS = {filt: IMG_DIR / f'ESO137-001-F{filt}W_drc.fits' for filt in FILTERS}

In [ ]:
# Get the Old WCS
OLD_WCS = WCS(fits.getheader(OLD_475_IMG_FN, 'SCI'))

# Get the New WCS
NEW_WCS = WCS(fits.getheader(IMG_FNS[475], 'SCI'))

In [ ]:
# Units
distToSrc = 69.6 * u.Mpc
MAX_SEP = 0.2 * u.arcsec

# Uncertainty Samples
N_SAMPLES = int(1e5)

# Median Av
CORRECTION_FACTOR = 0.44
MED_AV = CORRECTION_FACTOR*unc.normal(0.72, std=0.2, n_samples=N_SAMPLES)

# Image Scale
_ = u.set_enabled_equivalencies(
    u.pixel_scale(0.03 * u.arcsec / u.pixel) +
    u.pixel_scale(101.93679918450562 * u.pixel / u.kpc)
)

# Valid Galaxy Sources
# These are defined in the OLD WCS and are converted to the NEW WCS
VALID_GALAXY_COORDS = SkyCoord.from_pixel(*SkyCoord(
    ra=[243.3638605, 243.3640097, 243.3641405, 243.3631407, 243.3632252],
    dec=[-60.7633313, -60.7641447, -60.7643351, -60.7645508, -60.7648934],
    unit=(u.deg, u.deg),
    frame='fk5'
).to_pixel(OLD_WCS), wcs=NEW_WCS)

In [ ]:
ESO_ZP_TABLE = QTable.read('001-ZeroPointTable.ecsv')
ESO_ZP_TABLE

## Load the MUSE Data

In [ ]:
# Load in the Fossati EWs
museTable = pd.read_json(MUSE_FN)

# Get Coordinates
museCrd = SkyCoord(
    ra=museTable['RA'],
    dec=museTable['DEC'],
    unit=u.deg,
    frame='fk5'
)

# Set NaN Av to 0
museTable.loc[np.isnan(museTable['Av']), 'Av'] = 0
museTable.loc[np.isnan(museTable['Av_err']), 'Av_err'] = 0

# Get MUSE Luminosity
fHaUnit     = 1e-20*u.erg/u.s/u.cm**2
toLuminUnit = 4*np.pi*distToSrc**2
museLumin = unc.normal(
    museTable['fHa_cor'].values,
    std=museTable['fHa_cor_err'].values,
    n_samples=N_SAMPLES
)*fHaUnit*toLuminUnit
museLumin <<= (u.erg/u.s)

## Load SSP Models

In [ ]:
# Load in Starburst99 and Mappings
sbMod = Starburst99Spectrum(
    str(SB99_FN), redshift=0.0163, distToSrc=distToSrc.to_value('cm')
)
clMod = CloudySpectrum(
    str(CLDY_GLOB), redshift=0.0163, distToSrc=distToSrc.to_value('cm')
)

# Trim Sb99 and Make Composite Timeline
stInd = np.flatnonzero(sbMod.years > 1e7)[0]  # 1e7 is ending year of Cloudy Model
sbMod.years = sbMod.years[stInd:]
sbMod.spectrumList = sbMod.spectrumList[stInd:]
model = clMod + sbMod

## Interpolation
# yrs = 1e6*np.arange(1, 999, 0.1) * u.yr
yrs = np.geomspace(1, 998, 1000) * u.Myr
yrs = yrs.to('yr')

In [ ]:
# Load Filters
filts = {
    275:ObsBandpass('wfc3,uvis2,f275w'),
    475:ObsBandpass('acs,wfc1,f475w'  ),
    814:ObsBandpass('acs,wfc1,f814w'  )
}

# Get the Magnitude for Each Filter
trkMags = {}
for filt in filts:
    trkMags[filt] = model.makeobservation(filts[filt])

In [ ]:
# Plot the Track for Assurance
plt.close('all')
fig, ax = plt.subplots()
_ = ax.plot(
    trkMags[275] - trkMags[475],
    trkMags[475] - trkMags[814]
)

## Load Catalogs

In [ ]:
# Get the Single Image Catalogs
singleImgCats = {
    filt: QTable.read(CAT_DIR / f'ESO_F{filt}WxF{filt}W.cat', format='ascii.sextractor')
    for filt in FILTERS
}

# Get the Detection Coordinates
detCrds = {
    filt: SkyCoord(
        ra=cat['X_WORLD'],
        dec=cat['Y_WORLD'],
        frame='fk5'
    )
    for filt, cat in singleImgCats.items()
}

# Get the Dual Image Catalogs
# Use F475W as the detection image
dualImgCats = {
    filt: QTable.read(CAT_DIR / f'ESO_F475WxF{filt}W.cat', format='ascii.sextractor')
    for filt in FILTERS
}

### Trim by Correlation

Now, we only want to keep (in the dual catalogs) the two and three band detects.

In [ ]:
# Get the F275W/F475W Correlated Sources
idx275, sep, _ = detCrds[275].match_to_catalog_sky(detCrds[475])
idx275 = idx275[sep <= MAX_SEP]

# Get the F814W/F475W Correlated Sources
idx814, sep, _ = detCrds[814].match_to_catalog_sky(detCrds[475])
idx814 = idx814[sep <= MAX_SEP]

# The Intersection of the Two are the Three Band Detections
idx3Band = np.intersect1d(idx275, idx814)

# Get the Two Band Detections that are not in the Three Band Detections
idx2Lower = np.setdiff1d(idx275, idx3Band)
idx2Upper = np.setdiff1d(idx814, idx3Band)

In [ ]:
# Mark these Correlations in the F475W catalog
dualImgCats[475]['CORR'] = np.zeros(len(dualImgCats[475]), dtype=int)
dualImgCats[475]['CORR'][idx3Band] = 275+475+814
dualImgCats[475]['CORR'][idx2Lower] = 275+475
dualImgCats[475]['CORR'][idx2Upper] = 475+814

# Get/Keep the Union of all Correlated Sources
allCorrInds = reduce(np.union1d, [idx3Band, idx2Lower, idx2Upper])
for filt, cat in dualImgCats.items():
    dualImgCats[filt] = cat[allCorrInds]

### Remove MW Stars

In [ ]:
# Get the Source Coordinates
srcCrds = SkyCoord(
    ra=dualImgCats[475]['X_WORLD'],
    dec=dualImgCats[475]['Y_WORLD'],
    frame='fk5'
)

# Get the MW Star Locations
mwStarCrds = SkyCoord([
    reg.center for reg in regions.Regions.read(GAIA_REG_FILE)
])

# For the Source Coordinates and the Catalogs
# Only Keep those that are not MW Stars
# Only Keep things that are not within 15 pixels of a MW Star
_, sep, _ = srcCrds.match_to_catalog_sky(mwStarCrds)
srcCrds = srcCrds[sep > 15 * u.pix]
dualImgCats = {
    filt: cat[sep > 15 * u.pix] for filt, cat in dualImgCats.items()
}

### Remove by Brightness & Color

In [ ]:
# Fix the MAG_APER Columns
for filt, cat in dualImgCats.items():

    # Get the Filter Row
    row = ESO_ZP_TABLE[ESO_ZP_TABLE['index'] == filt]

    # Get the New Mag
    cat['MAG_APER_AB']  = u.Magnitude(cat['FLUX_APER']) + row['ZP_AB']
    cat['MAG_APER_AB'] -= row['GalExt'] + row['ApCor'] + row['KCor']

    # Get the New Mag Err
    cat['MAGERR_APER_AB']  = 2.5 / np.log(10) * cat['FLUXERR_APER'] / cat['FLUX_APER']
    cat['MAGERR_APER_AB'] /= row['DrizCor']
    cat['MAGERR_APER_AB']  = cat['MAGERR_APER_AB'].value << u.mag
    cat['MAGERR_APER_AB'][cat['FLUXERR_APER'] > cat['FLUX_APER']] = np.nan

In [ ]:
# Set the Magnitude Cutoffs
MAG_CUTS = {
    475: 20.45 - 1 << u.AB,
    814: 20.4 << u.AB,  # Based on Color-Mag Diagram
}

# Apply the Magnitude Cutoffs
# Things that are "True" are kept
keepMask  = (dualImgCats[475]['MAG_APER_AB'] > MAG_CUTS[475])
keepMask &= (dualImgCats[814]['MAG_APER_AB'] > MAG_CUTS[814])

In [ ]:
# Set the Color Cutoffs
COLOR_CUTS = {
    275-475: (2.24 + 0.66) * u.mag,
    475-814:  2.00        * u.mag
}

# Apply the Color Cutoffs
color = dualImgCats[475]['MAG_APER_AB'] - dualImgCats[814]['MAG_APER_AB']
keepMask &= (color < COLOR_CUTS[475-814])
color = dualImgCats[275]['MAG_APER_AB'] - dualImgCats[475]['MAG_APER_AB']
keepMask &= (color < COLOR_CUTS[275-475])

In [ ]:
# Some Valid Galaxy Sources get removed above
# This set of lines adds those sources back to the keepMask
idx, sep, _ = srcCrds.match_to_catalog_sky(VALID_GALAXY_COORDS)
keepMask |= (sep < (10*u.pix))

In [ ]:
# We also want to keep any sources that are inside H2 Regions
# The sun-HII-img.reg file called below matches the regions used in the paper
# (eso137_001_HII_region.reg). I use the Image coords file so I dont have to
# switch between WCSs
h2Regs = regions.Regions.read(OLD_REG_DIR / 'eso137_001_HII_region.reg')
for reg in h2Regs:
    keepMask |= reg.contains(srcCrds, NEW_WCS)

In [ ]:
# Adding Back the HII regions is over-generous. This removes some of the
# stars in the HII regions.
h2StarsRegs = regions.Regions.read(OLD_REG_DIR / 'H2-Exceptions-img.reg')
srcCrdsPix = regions.PixCoord.from_sky(srcCrds, NEW_WCS)
for reg in h2StarsRegs:
    keepMask &= ~reg.contains(srcCrdsPix)

In [ ]:
# Finally Apply the Mask
srcCrds = srcCrds[keepMask]
dualImgCats = {filt: cat[keepMask] for filt, cat in dualImgCats.items()}

### Remove Sources High/Bad Error Sources

Any sources that have error > 1 mag or NaN error are removed.

In [ ]:
# Max Error
MAX_ERROR = 1 << u.mag

# Get the New Mask
keepMask  = (dualImgCats[275]['MAGERR_APER_AB'] <= MAX_ERROR)
keepMask &= (dualImgCats[475]['MAGERR_APER_AB'] <= MAX_ERROR)
keepMask &= (dualImgCats[814]['MAGERR_APER_AB'] <= MAX_ERROR)

# Apply the Mask
srcCrds = srcCrds[keepMask]
dualImgCats = {filt: cat[keepMask] for filt, cat in dualImgCats.items()}

### Fix SExtractor Sorting Issues

For some reason, in dual image mode, sources do not get put in the same place
in the catalog as expected. This just seeks to reorganize the catalogs if needed.

In [ ]:
# Loop through Filters
keepMask   = np.ones_like(srcCrds, dtype=bool)
properSort = np.arange(len(srcCrds))
for filt, cat in dualImgCats.items():

    # Skip F475W
    if filt == 475:
        continue

    # Get the Coords for the Other Filter
    crds = SkyCoord(
        ra=cat['X_WORLD'],
        dec=cat['Y_WORLD'],
        frame='fk5'
    )

    # Match the Coords to the F475W Coords
    idxOther, sep, _ = srcCrds.match_to_catalog_sky(crds)

    # Drop the Affected Indices if Not Sorted
    if not is_sorted_no_repeats(idxOther):
        keepMask &= (idxOther == properSort)

    # Drop Sources with Large Separation
    SORT_SEP = 0.1 * u.arcsec
    keepMask &= (sep < SORT_SEP)

# Drop the Badly Sorted Sources
srcCrds = srcCrds[keepMask]
dualImgCats = {filt: cat[keepMask] for filt, cat in dualImgCats.items()}

### Only Keep HII Region Sources

In [ ]:
# Assume None are in the HII Regions
keepMask = np.zeros_like(srcCrds, dtype=bool)
h2Regs = regions.Regions.read(OLD_REG_DIR / 'eso137_001_HII_region.reg')
for reg in h2Regs:
    keepMask |= reg.contains(srcCrds, NEW_WCS)

# Only Keep HII Region Sources
srcCrds = srcCrds[keepMask]
dualImgCats = {filt: cat[keepMask] for filt, cat in dualImgCats.items()}

## Calculations

### Extinction

In [ ]:
# Setup the Mag Reddening
A_v = CORRECTION_FACTOR*unc.normal(
    museTable['Av'], std=museTable['Av_err'],
    n_samples=N_SAMPLES
) << u.mag

# Get the Extinction Based on the Reddening
RVP = 4.05
# RVP = 3.10
ebv = A_v/cal00(0.551, rvp=RVP)[0]

# Get the Magnitude Reddening in Each Filter
red, medRed = {}, {}
_, red[275] = cal00(0.275, ebv, rvp=RVP)
_, red[475] = cal00(0.475, ebv, rvp=RVP)
_, red[814] = cal00(0.814, ebv, rvp=RVP)

### Get the ESO Mags and Colors

In [ ]:
# Get the Mags
esoMags = {
    filt: unc.normal(
        cat['MAG_APER_AB'],
        std=cat['MAGERR_APER_AB'],
        n_samples=N_SAMPLES
    )
    for filt, cat in dualImgCats.items()
}

In [ ]:
# Apply Reddening based on MUSE
idx, _, _ = srcCrds.match_to_catalog_sky(museCrd)

# Setup Extinction
r2, r4, r8 = red[275][idx], red[475][idx], red[814][idx]

# Mags
esoX = (esoMags[275] - esoMags[475]) - (r2 - r4)
esoY = (esoMags[475] - esoMags[814]) - (r4 - r8)

In [ ]:
# Get the Track Color Info
trkX = trkMags[275] - trkMags[475]
trkY = trkMags[475] - trkMags[814]

# Interpolate the Tracks
trk4 = interp1d(model.years, trkMags[475], kind='linear')(yrs)  << u.mag
trkX = interp1d(model.years, trkX, kind='linear')(yrs)          << u.mag
trkY = interp1d(model.years, trkY, kind='linear')(yrs)          << u.mag

### Estimate the Age

In [ ]:
# Get the Age Estimates
ageInds, ageEsts = unc_euclidean_inds(esoX.value, esoY.value, trkX.value, trkY.value, yrs)

In [ ]:
# Get the Mass Estimates
trkMag = trk4[ageInds]

# Get Match
ind, sep, _ = srcCrds.match_to_catalog_sky(museCrd)

# Setup Extinction
r4 = red[475][ind]

# Get the Mag Offset
# Note that the track mag is 10^6 solar mag
magOffset = (esoMags[475] - r4).distribution - trkMag

# Get the Mass Est
massEsts = unc.Distribution(10**(6 - magOffset.value/2.5)) * u.Msun